In [1]:
import numpy as np
import time
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
device

device(type='cpu')

In [3]:
def puf_query(c, w):
    n = c.shape[1]
    phi = np.ones(n+1)
    phi[n] = 1
    for i in range(n-1, -1, -1):
        phi[i] = (2*c[0,i]-1)*phi[i+1]

    r = (np.dot(phi, w) > 0)
    return r

In [4]:
# Problem Setup
target = 0.99  # The desired prediction rate
n = 64  # number of stages in the PUF

In [5]:
# Initialize the PUF
np.random.seed(int(time.time()))
data = np.loadtxt('weight_diff.txt')
w = np.zeros((n+1, 1))
for i in range(1, n+2):
    randi_offset = np.random.randint(1, 45481)
    w[i-1] = data[randi_offset-1]

In [6]:
# Syntax to query the PUF:
c = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
r = puf_query(c, w)
# you may remove these two lines

In [7]:
# Generating the training set #ENTER TRAINING SIZE HERE FOR DIFFERENT TRAILS
training_size = 5000  # Number of training samples
X_train = np.random.randint(0, 2, size=(training_size, n))  # Random challenge vectors
y_train = np.zeros((training_size, 1))  # Response bits
for i in range(training_size):
    y_train[i] = puf_query(X_train[i].reshape(1, -1), w)

In [8]:
X_train

array([[1, 1, 1, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1]])

In [9]:
y_train

array([[1.],
       [0.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]])

In [10]:
# Calculating Phi
def calc_phi(select_bits):
    phi_vals = []
    for i in range(len(select_bits)):
        target_slice = select_bits[i:]
        zeros = [z for z in target_slice if z == 0]
        phi = 1 if len(zeros) % 2 == 0 else -1
        phi_vals.append(phi)
    return np.array(phi_vals + [1])


X_train_phi = np.apply_along_axis(calc_phi, 1, X_train)

In [11]:
X_train_phi

array([[-1, -1, -1, ..., -1,  1,  1],
       [ 1,  1,  1, ..., -1, -1,  1],
       [-1,  1,  1, ...,  1, -1,  1],
       ...,
       [ 1, -1,  1, ...,  1, -1,  1],
       [-1, -1,  1, ..., -1, -1,  1],
       [-1,  1, -1, ...,  1,  1,  1]])

In [12]:
# Sigmoid function
def sigmoid(z):
    return torch.sigmoid(torch.tensor(z)).numpy()

In [13]:
# Cost function
def cost(phi, theta, y):
    z = phi @ theta
  
    epsilon =  0.001
    cost0 = y.T.dot(np.log(sigmoid(z) + epsilon))
    cost1 = (1-y).T.dot(np.log(1-sigmoid(z) + epsilon))
    cost = -((cost1 + cost0))/len(y) 
    return cost

In [14]:
# Logistic regression #ADJUST LEARNING RATE AND ITERATIONS FOR EACH TRAIL (THIS CELL AND CELL BELOW)
def logistic_regression(X, y, learning_rate=0.01, num_iterations=500):
    m, n = X.shape
    w = np.random.randn(n)
    w = w.reshape(-1, 1)
    phi = X

    for i in range(num_iterations):
        loss = cost(phi, w, y)
        z = np.dot(phi, w)
        h = sigmoid(z)
  
        np.dot(phi.T, h - np.reshape(y,(len(y),1)))
        w = w - learning_rate * np.dot(phi.T, sigmoid(np.dot(phi, w)) - np.reshape(y,(len(y),1)))

        print("Iteration {}/{} - Loss: {}".format(i+1, num_iterations, loss))
    return w

In [15]:
# Estimating w and timed trials
w0 = np.zeros((n+1, 1))  # The estimated value of w.
t0 = time.process_time()
w0 = logistic_regression(X_train_phi, y_train, learning_rate=0.01, num_iterations=500)
w0 = w0.reshape(-1)
t1 = time.process_time()
training_time = t1 - t0  # time taken to get w0
print("Training time:", training_time)
print("Training size:", training_size)

Iteration 1/500 - Loss: [[2.74279196]]
Iteration 2/500 - Loss: [[0.75511166]]
Iteration 3/500 - Loss: [[0.11056461]]
Iteration 4/500 - Loss: [[0.03410826]]
Iteration 5/500 - Loss: [[0.02535872]]
Iteration 6/500 - Loss: [[0.02357001]]
Iteration 7/500 - Loss: [[0.02304507]]
Iteration 8/500 - Loss: [[0.0228508]]
Iteration 9/500 - Loss: [[0.022753]]
Iteration 10/500 - Loss: [[0.02268678]]
Iteration 11/500 - Loss: [[0.02263204]]
Iteration 12/500 - Loss: [[0.0225819]]
Iteration 13/500 - Loss: [[0.02253381]]
Iteration 14/500 - Loss: [[0.02248678]]
Iteration 15/500 - Loss: [[0.0224404]]
Iteration 16/500 - Loss: [[0.02239449]]
Iteration 17/500 - Loss: [[0.02234898]]
Iteration 18/500 - Loss: [[0.02230383]]
Iteration 19/500 - Loss: [[0.02225901]]
Iteration 20/500 - Loss: [[0.02221453]]
Iteration 21/500 - Loss: [[0.02217036]]
Iteration 22/500 - Loss: [[0.02212651]]
Iteration 23/500 - Loss: [[0.02208297]]
Iteration 24/500 - Loss: [[0.02203974]]
Iteration 25/500 - Loss: [[0.02199681]]
Iteration 26/5

Iteration 218/500 - Loss: [[0.01689119]]
Iteration 219/500 - Loss: [[0.01687421]]
Iteration 220/500 - Loss: [[0.01685729]]
Iteration 221/500 - Loss: [[0.01684042]]
Iteration 222/500 - Loss: [[0.01682362]]
Iteration 223/500 - Loss: [[0.01680687]]
Iteration 224/500 - Loss: [[0.01679018]]
Iteration 225/500 - Loss: [[0.01677354]]
Iteration 226/500 - Loss: [[0.01675696]]
Iteration 227/500 - Loss: [[0.01674043]]
Iteration 228/500 - Loss: [[0.01672396]]
Iteration 229/500 - Loss: [[0.01670754]]
Iteration 230/500 - Loss: [[0.01669118]]
Iteration 231/500 - Loss: [[0.01667487]]
Iteration 232/500 - Loss: [[0.01665862]]
Iteration 233/500 - Loss: [[0.01664242]]
Iteration 234/500 - Loss: [[0.01662627]]
Iteration 235/500 - Loss: [[0.01661018]]
Iteration 236/500 - Loss: [[0.01659413]]
Iteration 237/500 - Loss: [[0.01657814]]
Iteration 238/500 - Loss: [[0.0165622]]
Iteration 239/500 - Loss: [[0.01654632]]
Iteration 240/500 - Loss: [[0.01653048]]
Iteration 241/500 - Loss: [[0.0165147]]
Iteration 242/500 

Iteration 434/500 - Loss: [[0.01416835]]
Iteration 435/500 - Loss: [[0.01415887]]
Iteration 436/500 - Loss: [[0.01414941]]
Iteration 437/500 - Loss: [[0.01413997]]
Iteration 438/500 - Loss: [[0.01413055]]
Iteration 439/500 - Loss: [[0.01412116]]
Iteration 440/500 - Loss: [[0.01411178]]
Iteration 441/500 - Loss: [[0.01410242]]
Iteration 442/500 - Loss: [[0.01409309]]
Iteration 443/500 - Loss: [[0.01408377]]
Iteration 444/500 - Loss: [[0.01407448]]
Iteration 445/500 - Loss: [[0.0140652]]
Iteration 446/500 - Loss: [[0.01405594]]
Iteration 447/500 - Loss: [[0.01404671]]
Iteration 448/500 - Loss: [[0.01403749]]
Iteration 449/500 - Loss: [[0.01402829]]
Iteration 450/500 - Loss: [[0.01401911]]
Iteration 451/500 - Loss: [[0.01400995]]
Iteration 452/500 - Loss: [[0.01400081]]
Iteration 453/500 - Loss: [[0.01399169]]
Iteration 454/500 - Loss: [[0.01398259]]
Iteration 455/500 - Loss: [[0.01397351]]
Iteration 456/500 - Loss: [[0.01396445]]
Iteration 457/500 - Loss: [[0.0139554]]
Iteration 458/500 

In [16]:
w0 = w0.reshape(-1)

In [17]:
w0.shape

(65,)

In [18]:
# Results
n_test = 10000
correct = 0
for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
    r = puf_query(c_test, w)
    r0 = puf_query(c_test, w0)
    correct += (r==r0)

success_rate = correct/n_test
print("Success rate:", success_rate)

# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
    effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)





Success rate: [0.9927]
Effective training time: 0.046875
